# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [1]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [2]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [3]:
def astar(problem: Problem):  
    queue = []
    explored = []
    node= [problem.heuristic(problem.initial_state),0,problem.initial_state,[]]
    queue.append(node)
    while queue:
        node = queue.pop(0)
        if node[2] in explored:
            continue
        explored.append(node[2])
        if problem.is_goal(node[2]):

            return node[3],len(explored)
        actions = problem.available_actions(node[2])
        for action in actions:
            new_node = problem.do_action(node[2],action)
            if new_node not in explored:  
                cost = node[1]+problem.action_cost(node[1],action)
                queue.append([cost+problem.heuristic(new_node),cost,new_node,node[3] + [action]])
                queue = sorted(queue, key=lambda x: x[0])


In [4]:
%timeit astar(VacuumProblem())

15.8 µs ± 3.88 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Now lets test your code in the vacuum world!

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [5]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
print(astar(VacuumProblem1()))

(['Suck', 'Right', 'Suck'], 7)


In [6]:
%timeit astar(VacuumProblem1())

17.2 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


And another in which heuristic grossly overestimates the cost.

In [7]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
print(astar(VacuumProblem2()))

(['Suck', 'Right', 'Suck'], 4)


In [8]:
%timeit astar(VacuumProblem2())

8.99 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

At first glance one could state that the second algorithm is the best, yet after taking a longer look at how the algorithm calculates the heuristics, it becomes obvious that the algorithm overestimates the the cost. This fault makes this heuristics unusable in other problems.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [9]:
import copy
class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        return [[7,2,4], [5,0,6], [8,3,1]]

    def available_actions(self, state):
        return ['Up', 'Right', 'Down', 'Left']

    def getIndex(self,state):
        for x, y in enumerate(state):
            if 0 in y:
                row = x
                col = y.index(0)
                return (row,col)
    def do_action(self, state, action):
        row,col = self.getIndex(state)
        new_state = copy.deepcopy(state)
        if action == 'Up':
            if row == 0:
                return state
            else:
                new_state[row][col] = new_state[row - 1][col]
                new_state[row - 1][col] = 0
            
        elif action == 'Down':
            if row == 2:
                return state
            else:
                new_state[row][col] = state[row + 1][col]
                new_state[row + 1][col] = 0

        elif action == 'Right':
            if col == 2:
                return state
            else:
                new_state[row][col] = state[row][col + 1]
                new_state[row][col + 1] = 0

        elif action == 'Left':
            if col == 0:
                return state
            else:
                new_state[row][col] = state[row][col - 1]
                new_state[row][col - 1] = 0
        return new_state

    def is_goal(self, state):
        if state == [[0,1,2],[3,4,5], [6,7,8]]:
            return True
        else:
            return False
        
    def action_cost(self, state, action) -> float:
        return 1
    def manhatan_distance(self, state) -> float:  
        distance = 0
        for number in range(9):
            x1,y1 = coordinates(state,number)
            x2,y2 = coordinates([[0,1,2],[3,4,5], [6,7,8]],number)            
            distance += abs(x1-x2) + abs(y1-y2)
        return distance
    def euclidean_distance(self, state) -> float: 
        distance = 0
        for number in range(9):
            x1,y1 = coordinates(state,number)
            x2,y2 = coordinates([[0,1,2],[3,4,5], [6,7,8]],number)  
            distance += sqrt((x1-x2)**2+(y1-y2)**2)
        return distance
    def heuristic(self, state) -> float:
        return self.manhatan_distance(state)

**Prove that this heuristic is admissible.**

Knowing that distance between a tile and its neighbours is equal to 1, and every possible action may reduce the distance by one lead to the conclusion that the manhatan distance is always (for square grid) smaller or equal to the real distance, which proves the consistency of the heuristics.


In [10]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:    
        return self.euclidean_distance(state)

In [11]:
def coordinates(state,number):
    for i in state:
        if number in i:
            y = state.index(i)
    for i in list(zip(*state)):
        if number in i:
            x = list(zip(*state)).index(i)
    return x,y

**Prove that this heuristic is admissible.**

Using aforementioned proof it is enough to observe that the Euclidean distance is always less than or equal to manhatan distance.


In [12]:
from math import sqrt

In [13]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        mhd,ed = self.manhatan_distance(state),self.euclidean_distance(state)
        return (mhd+ed)/2

**Prove that this heuristic is admissible.**

Like above, the average of long and short distance is always smaller than the long one.

Run your heuristics on the given problem.

In [14]:
plan0 = astar(PuzzleProblem())
print(plan0)
plan1 = astar(PuzzleProblem1())
print(plan1)
plan2 = astar(PuzzleProblem2())
print(plan2)

print("Is plan0==plan1?", plan0[0] == plan1[0])
print("Is plan0==plan2?", plan0[0] == plan2[0])
print("Is plan1==plan2?", plan1[0] == plan2[0])

(['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up'], 3825)
(['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up'], 8438)
(['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up'], 5287)
Is plan0==plan1? True
Is plan0==plan2? True
Is plan1==plan2? True


**Which of the heuristics is the best for this task? Why is that?**

It can be easily seen that manhatan distance is the best. It always approximate cost better which results in reducing number of visited states by more than a half. It is caused by the fact that manhatan distance is always greater (or equal) to both euclidean distance, and average of both distances

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.